# Music Recomendations

In [ ]:
!pip install catboost==1.0.3
!pip install gensim==4.1.0
!pip install --upgrade numpy

In [23]:
!nvidia-smi

Tue Nov 23 22:54:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    75W / 149W |     70MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [24]:
%cd /content
!wget https://voudy-data.s3.eu-north-1.amazonaws.com/wsdm_data.tar.gz
!tar -xf /content/wsdm_data.tar.gz
!mkdir data
!mv -t data members.csv song_extra_info.csv songs.csv train.csv
!git clone -b hw2 https://github.com/Sushentsev/recommendation-systems
%cd recommendation-systems/
!mv /content/data /content/recommendation-systems/hw2

/content
--2021-11-23 22:54:49--  https://voudy-data.s3.eu-north-1.amazonaws.com/wsdm_data.tar.gz
Resolving voudy-data.s3.eu-north-1.amazonaws.com (voudy-data.s3.eu-north-1.amazonaws.com)... 52.95.170.32
Connecting to voudy-data.s3.eu-north-1.amazonaws.com (voudy-data.s3.eu-north-1.amazonaws.com)|52.95.170.32|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 613394184 (585M) [application/x-gzip]
Saving to: ‘wsdm_data.tar.gz.2’

wsdm_data.tar.gz.2  100%[===================>] 584.98M  88.3MB/s    in 6.8s    

2021-11-23 22:54:56 (85.4 MB/s) - ‘wsdm_data.tar.gz.2’ saved [613394184/613394184]

mkdir: cannot create directory ‘data’: File exists
fatal: destination path 'recommendation-systems' already exists and is not an empty directory.
/content/recommendation-systems
mv: cannot move '/content/data' to '/content/recommendation-systems/hw2/data': Directory not empty


In [5]:
%cd /content/recommendation-systems/hw2

/content/recommendation-systems/hw2


In [6]:
import sys
sys.path.append("/content/recommendation-systems")

In [7]:
from datasets.members import MembersDataset
from datasets.songs import SongsDataset
from datasets.train import TrainDataset
from datasets.songs_info import SongsInfoDataset

from models.catboost_model import CatBoostModel
from models.embeddings_model import EmbeddingModel

from embeddings_builder import EmbeddingsBuilder

from plot import * 
from utils import * 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Подготовка данных

In [3]:
random_state = 42

Загрузим данные в память. 

In [8]:
%%time
members_dataset = MembersDataset.from_path("./data/members.csv")
songs_dataset = SongsDataset.from_path("./data/songs.csv")
train_dataset = TrainDataset.from_path("./data/train.csv")
songs_info_dataset = SongsInfoDataset.from_path("./data/song_extra_info.csv")

CPU times: user 55.4 s, sys: 1.32 s, total: 56.7 s
Wall time: 56.9 s


Построим признаки для песен и пользователей и посмотрим, что получилось. 

In [9]:
%%time
members_dataset = members_dataset.create_features()
songs_dataset = songs_dataset.create_features(songs_info_dataset)

CPU times: user 11.4 s, sys: 590 ms, total: 12 s
Wall time: 12 s


In [10]:
members_dataset.pandas_df.head(3)

,msno,city,gender,registered_via,bd_category,registration_init_year,expiration_date_year
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,<UNK>,7,<UNK>,2011,2017
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,<UNK>,7,<UNK>,2015,2017
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,<UNK>,4,<UNK>,2016,2017


In [11]:
songs_dataset.pandas_df.head(3)

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language,genres_count,artist_name_count,composer_count,lyricists_count,isrc_year
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0,1,1,1,1,2014
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0,1,1,3,1,-1
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,<UNK>,<UNK>,31.0,1,1,0,0,-1


Теперь добавим в обучающую выборку признаков из песен и пользователей. 

In [12]:
train_dataset = train_dataset.merge(members_dataset, on="msno", how="left")
train_dataset = train_dataset.merge(songs_dataset, on="song_id", how="left")

In [13]:
train_dataset.pandas_df.head(3)

,msno,song_id,source_system_tab,source_screen_name,source_type,target,city,gender,registered_via,bd_category,...,genre_ids,artist_name,composer,lyricist,language,genres_count,artist_name_count,composer_count,lyricists_count,isrc_year
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,1,<UNK>,7,<UNK>,...,359,Bastille,Dan Smith| Mark Crew,<UNK>,52.0,1.0,1.0,2.0,0.0,2016
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1,13,female,9,young,...,1259,Various Artists,<UNK>,<UNK>,52.0,1.0,1.0,0.0,0.0,1999
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1,13,female,9,young,...,1259,Nas,N. Jones、W. Adams、J. Lordan、D. Ingle,<UNK>,52.0,1.0,1.0,1.0,0.0,2006


Теперь преобразуем некоторые колонки категориальным значениям и заполнил пропуски. 

In [14]:
train_dataset.to_category(["msno", "song_id"])
train_dataset.fill_na_category(["source_system_tab", "source_screen_name", "source_type"])

In [15]:
train_dataset.pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7377418 entries, 0 to 7377417
Data columns (total 23 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   msno                    category
 1   song_id                 category
 2   source_system_tab       category
 3   source_screen_name      category
 4   source_type             category
 5   target                  uint8   
 6   city                    category
 7   gender                  category
 8   registered_via          category
 9   bd_category             category
 10  registration_init_year  int64   
 11  expiration_date_year    int64   
 12  song_length             float64 
 13  genre_ids               category
 14  artist_name             category
 15  composer                category
 16  lyricist                category
 17  language                category
 18  genres_count            float64 
 19  artist_name_count       float64 
 20  composer_count          float64 
 21  lyricist

Все круто!

In [16]:
na_mask = train_dataset.pandas_df.isna().any(axis=1)
print(f"Количество записей, для которых не нашлось пользователя или песни: {sum(na_mask)}")

Количество записей, для которых не нашлось пользователя или песни: 114


Их оказалось не так много, уберем из нашего набора данных.

In [17]:
train_dataset = train_dataset.remove_by_mask(na_mask)

Так как **CatBoost** на CPU обучается достаточно долго (больше часа), то было решено обучать на GPU. Однако GPU не поддерживает группы больше 1023, поэтому тренировочный набор данных был уменьшен. При этом, желательно сохранить порядок записей.

In [18]:
train_dataset = train_dataset.reduce_by_members(size=1023)

In [19]:
train_dataset.pandas_df.target.value_counts()

1    3564182
0    3451109
Name: target, dtype: int64

Соотношение классов осталось тем же, все хорошо. Изменение размеров: 7377304 -> 7015291

## Задание 1

Построить рекомендации для каждого пользователя, произвести оценку качества с помощью 5-fold CV с метриками NDCG, ROC AUC per user.

При разработке признаков я умеренно не стал брать признаки, полученные с помощью матричных факторизаций, чтобы потом добавить неклассические эмбеддинги, полученные в задании 2. 

Для **CatBoost** необоходимо, чтобы данные были упорядочены по **queries**, поэтому будем обучать модели на упорядоченных данных, но эмбеддинги составлять на основе натурального порядка. 

In [21]:
train_dataset_sorted = train_dataset.sort_by("msno")
model = CatBoostModel("YetiRank", 15, "CPU", random_state)
scores = model.cv_scores(train_dataset_sorted, n_splits=5)

Train dataset size: 5612232 | Test dataset size: 1403059
Groupwise loss function. OneHotMaxSize set to 10
0:	total: 16.4s	remaining: 3m 49s
1:	total: 30.8s	remaining: 3m 20s
2:	total: 45.9s	remaining: 3m 3s
3:	total: 1m 1s	remaining: 2m 48s
4:	total: 1m 15s	remaining: 2m 30s
5:	total: 1m 29s	remaining: 2m 14s
6:	total: 1m 43s	remaining: 1m 58s
7:	total: 1m 58s	remaining: 1m 43s
8:	total: 2m 12s	remaining: 1m 28s
9:	total: 2m 26s	remaining: 1m 13s
10:	total: 2m 40s	remaining: 58.3s
11:	total: 2m 53s	remaining: 43.4s
12:	total: 3m 7s	remaining: 28.8s
13:	total: 3m 23s	remaining: 14.6s
14:	total: 3m 37s	remaining: 0us


/Users/Denis.Sushentsev/Study/recommendation-systems/hw2/metrics.py:12: RuntimeWarning: invalid value encountered in true_divide
  return sum(relevance / np.log2(1 + np.arange(n_items)))
/Users/Denis.Sushentsev/Study/recommendation-systems/hw2/metrics.py:22: RuntimeWarning: invalid value encountered in true_divide
  return sum(relevance / np.log2(1 + np.arange(n_items)))
/Users/Denis.Sushentsev/Study/recommendation-systems/hw2/metrics.py:12: RuntimeWarning: divide by zero encountered in true_divide
  return sum(relevance / np.log2(1 + np.arange(n_items)))
/Users/Denis.Sushentsev/Study/recommendation-systems/hw2/metrics.py:22: RuntimeWarning: divide by zero encountered in true_divide
  return sum(relevance / np.log2(1 + np.arange(n_items)))
/Users/Denis.Sushentsev/Study/recommendation-systems/hw2/metrics.py:37: RuntimeWarning: invalid value encountered in double_scalars
  ndcgs.append(query_dcg / query_idcg)


NDCG: nan | ROC AUC 0.6303940909554653
Train dataset size: 5612233 | Test dataset size: 1403058


KeyboardInterrupt: 

## Задание 2

Построить неклассические эмбеддинги и исследовать их полезность.

In [ ]:
%%time
embeddings = EmbeddingsBuilder(embedding_dim=100, random_state=random_state)
embeddings = embeddings.fit(train_df)
save_pickle(embeddings, "embeddings.pkl")